In [3]:
!pip install pdfplumber openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.6 MB/s eta 0:00:00


In [24]:
import os
from openpyxl import Workbook
import pdfplumber
import re
from datetime import datetime

directory= 'pdf_invoices'
files= os.listdir(directory)
files_quantity= len(files)

if files_quantity == 0:
    raise Exception("No files dound in the directory")

wb= Workbook()
ws= wb.active
ws.title= "Invoice Import"

ws["A1"]= "Order ID"
ws["B1"]= "Shipped Date"
ws["C1"]= "Client Name"
ws["D1"]= "File Name"
ws["E1"]= "Status"

last_empty_line=1
while ws["A" + str(last_empty_line)].value is not None:
    last_empty_line += 1

for file in files:
    file_path = os.path.join(directory, file)
    if os.path.isfile(file_path): # Check if the path points to a file
        with pdfplumber.open(file_path) as pdf:
            first_page= pdf.pages[0]
            pdf_text= first_page.extract_text()
            print(pdf_text)

        inv_number_re_pattern= r'Order ID: (\d+)'
        inv_date_re_pattern= r'Shipped Date: (\d{4}-\d{2}-\d{2})'
        inv_client_re_pattern= r'Ship Name: (.+)'

        match_number= re.search(inv_number_re_pattern, pdf_text)
        match_date= re.search(inv_date_re_pattern, pdf_text)
        match_client= re.search(inv_client_re_pattern, pdf_text)

        if match_number:
            invoice_number= match_number.group(1)
            ws['A{}'.format(last_empty_line)]= invoice_number
        else:
            ws['A{}'.format(last_empty_line)]= "Couln't find invoice number"

        if match_date:
            invoice_date= match_date.group(1)
            ws['B{}'.format(last_empty_line)]= invoice_date
        else:
            ws['B{}'.format(last_empty_line)]= "Couln't find invoice date"

        if match_client:
            invoice_client= match_client.group(1)
            ws['C{}'.format(last_empty_line)]= invoice_client
        else:
            ws['C{}'.format(last_empty_line)]= "Couln't find Client"

        ws['D{}'.format(last_empty_line)]= file
        ws['E{}'.format(last_empty_line)]= "Completed"

        last_empty_line +=1

full_now= str(datetime.now()).replace(":", "-")
dot_index= full_now.index(".")
now= full_now[:dot_index]

wb.save("Invoices - {}.xlsx".format(now))

Order ID: 10256
Shipping Details:
Ship Name: Wellington Importadora
Ship Address: Rua do Mercado, 12
Ship City: Resende
Ship Region: South America
Ship Postal Code: 08737-363
Ship Country: Brazil
Customer Details:
Customer ID: WELLI
Customer Name: Wellington Importadora
Employee Details:
Employee Name: Janet Leverling
Shipper Details:
Shipper ID: 2
Shipper Name: United Package
Order Details:
Order Date: 2016-07-15
Shipped Date: 2016-07-17
Products:
--------------------------------------------------------------------------------------------------
Product: Perth Pasties
Quantity: 15
Unit Price: 26.2
Total: 393.0
--------------------------------------------------------------------------------------------------
Product: Original Frankfurter grüne Soße
Quantity: 12
Unit Price: 10.4
Total: 124.80000000000001
Total Price:
Order ID: 10266
Shipping Details:
Ship Name: Wartian Herkku
Ship Address: Torikatu 38
Ship City: Oulu
Ship Region: Scandinavia
Ship Postal Code: 90110
Ship Country: Finland
